In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Pre-Processing
#### Preprocesses data from CSV files, drops specified columns and columns with missing values.


In [ ]:
# Load the dataset 'gee_features_10pct.csv' into a DataFrame called 'dt'
dt= pd.read_csv('/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features_10pct.csv')
# Load the sample submission CSV file into a DataFrame called 'smpl_sub'
smpl_sub = pd.read_csv('/kaggle/input/maternal-and-child-health-monitoring-in-lmics/sample submission.csv')

In [ ]:
smpl_sub

In [ ]:
# Merge the sample submission DataFrame 'smpl_sub' with the 'dt' DataFrame based on the 'DHSID' column
predict_df = pd.merge(smpl_sub, dt, on='DHSID')

In [ ]:
predict_df

In [ ]:
# List of columns to be dropped from 'dt' DataFrame
dt = dt.drop(['new_ind', 'index', 'ADM1DHS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1NAME', 'ADM1SALBCO', 'ADM1SALBNA', 'ADM1SALNA', 'ALT_DEM', 'ALT_GPS', 'CCFIPS', 'DATUM', 'DHSCC', 'DHSCLUST', 'DHSREGCO', 'DHSREGNA', 'DHSYEAR', 'F21', 'F22', 'F23', 'LATNUM', 'LONGNUM', 'SOURCE', 'URBAN_RURA', 'ZONECO', 'ZONENA','key1','key2','key3'], axis = 1)
predict_df = predict_df.drop(['Mean_BMI', 'Median_BMI', 'Unmet_Need_Rate', 'Under5_Mortality_Rate', 'Skilled_Birth_Attendant_Rate', 'Stunted_Rate','new_ind', 'index', 'ADM1DHS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1NAME', 'ADM1SALBCO', 'ADM1SALBNA', 'ADM1SALNA', 'ALT_DEM', 'ALT_GPS', 'CCFIPS', 'DATUM', 'DHSCC', 'DHSCLUST', 'DHSREGCO', 'DHSREGNA', 'DHSYEAR', 'F21', 'F22', 'F23', 'LATNUM', 'LONGNUM', 'SOURCE', 'URBAN_RURA', 'ZONECO', 'ZONENA','key1','key2','key3'], axis = 1)

In [ ]:
# Drop columns with missing values from the 'dt' DataFrame
dt = dt.dropna(axis = 1)
# Drop columns with missing values from the 'predict_df' DataFrame
predict_df = predict_df.dropna(axis = 1)

In [ ]:
# Extract 'DHSID' values from the 'dt' DataFrame
DHSID = np.array(dt['DHSID'])
# Extract 'DHSID' values from the 'predict_df' DataFrame
DHSID_sub = np.array(predict_df['DHSID'])

In [ ]:
predict_df

In [ ]:
# Drop the 'DHSID' column from the 'dt' DataFrame
dt = dt.drop('DHSID', axis = 1)
# Drop the 'DHSID' column from the 'predict_df' DataFrame
predict_df = predict_df.drop('DHSID', axis = 1)

In [ ]:
predict_df

## Principal Component Analysis (PCA) for Dimensionality Reduction
### Scaling the Data

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Scale the 'dt' dataset using StandardScaler
scaler = StandardScaler()
dt = scaler.fit_transform(dt)

# Initialize PCA
pca = PCA()

# Fit PCA to the scaled data
pca.fit(dt)

# Get explained variance ratios and cumulative variance ratios
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Plot the cumulative explained variance ratio
num_components = len(explained_variance_ratio)
plt.figure(figsize=(8, 6))
plt.plot(range(1, num_components + 1), cumulative_variance_ratio, marker='o', linestyle='-')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio vs. Number of Components')
plt.grid()
plt.show()

## Loading Training Labels
### Reading Training Labels CSV

In [ ]:
# Read the training labels CSV file into the 'df_train' DataFrame
df_train = pd.read_csv('/kaggle/input/maternal-and-child-health-monitoring-in-lmics/training_label.csv')

In [ ]:
df_train

In [ ]:
# Perform PCA with 2000 components on the 'dt' DataFrame
pca = PCA(n_components=2000)

# Transform the 'predict_df' DataFrame using a previously defined scaler
predict_sc = scaler.transform(predict_df)
predict_sc = pd.DataFrame(predict_sc)

# Apply PCA transformation to the 'dt' DataFrame
X_pca = pca.fit_transform(dt)

# Apply PCA transformation to the 'predict_df' DataFrame
X_pca_sub = pca.transform(predict_sc)

# Create DataFrames from the PCA-transformed data
transformed_df_final = pd.DataFrame(X_pca)
transformed_sub_final = pd.DataFrame(X_pca_sub)

# Add 'DHSID' columns to the PCA-transformed DataFrames
transformed_df_final['DHSID'] = DHSID
transformed_sub_final['DHSID'] = DHSID_sub

# Merge the 'df_train' DataFrame with the PCA-transformed DataFrames based on 'DHSID'
merged_df = pd.merge(df_train, transformed_df_final, on='DHSID')

In [ ]:
merged_df

In [ ]:
transformed_df_final

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# target columns
target_columns = ['Mean_BMI', 'Median_BMI', 'Unmet_Need_Rate', 'Under5_Mortality_Rate' ,'Skilled_Birth_Attendant_Rate' , 'Stunted_Rate']

# Create an empty dictionary to store RMSE values for each target column
rmse_dict = {}

for target_col in target_columns:
    merged_df_dropped = merged_df[merged_df[target_col].notna()]
    col_names = list(transformed_df_final.columns)
    col_names.remove('DHSID')
    X_value = merged_df_dropped[col_names]
    Y_value = merged_df_dropped[target_col]

    x_train, x_test, y_train, y_test = train_test_split(X_value, Y_value, test_size=0.2, random_state=42)
    len(x_train), len(x_test), len(y_train), len(y_test)

    elastic_net = ElasticNet()

    # Define a grid of hyperparameter values to search
    param_grid = {
        'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    }

    # Create GridSearchCV instance
    grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

    # Fit the GridSearchCV
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
    
    # Print the best hyperparameters and corresponding score
    print(f"Best Hyperparameters for {target_col}:", grid_search.best_params_)
    print(f"Best Score for {target_col}:", -grid_search.best_score_)
    
    preds = best_model.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    # Store RMSE in the dictionary
    rmse_dict[target_col] = rmse

# Calculate the average RMSE across all target columns
average_rmse = np.mean(list(rmse_dict.values()))

# Print the RMSE values for each target column and the average RMSE
for target_col, rmse in rmse_dict.items():
    print(f"RMSE for {target_col}: {rmse}")

print("Average RMSE across all columns:", average_rmse)

In [ ]:
average_rmse